# Introduction

This notebook is to study the SLALOM algorithm based on the original code in the [`slalom.py` script](https://github.com/mkanai/slalom/blob/master/slalom.py#L76) and the version we converted into R [slalom.R](https://github.com/StatFunGen/pecotmr/blob/main/R/slalom.R).

# Slalom in python

In [198]:
def abf(beta, se, W=0.04):
    z = beta / se
    V = se ** 2
    r = W / (W + V)
    lbf = 0.5 * (np.log(1 - r) + (r * z ** 2))
    denom = sp.special.logsumexp(lbf)
    prob = np.exp(lbf - denom)
    return lbf, prob

def get_cs(variant, prob, coverage=0.95):
    ordering = np.argsort(prob)[::-1]
    idx = np.where(np.cumsum(prob[ordering]) > coverage)[0][0]
    cs = variant[ordering][: (idx + 1)]
    return cs



In [199]:
import argparse
import os.path
import numpy as np
import scipy as sp
import pandas as pd
import scipy.stats as stats
pd.set_option('display.max_columns', None)

In [200]:
test_data_path = "/home/rd2972/private_data/20240300_rss_qc_imputation/SLALOM/"
z_file = test_data_path + "z_LD_dentist_test_sum.stat.txt"
LD_file = test_data_path + "z_LD_dentist_test_LD.txt"

# Read the data into pandas DataFrame
snp_data = pd.read_csv(z_file, sep = ' ')
LD_data = pd.read_csv(LD_file, sep = ' ')
snp_data["variant"] = snp_data["chr"].astype(str) + ":" + snp_data["pos"].astype(str) + ":" + snp_data["ref"] + ":" + snp_data["alt"]
snp_data["se"] = 1
snp_data["beta"] = snp_data["z"] * snp_data["se"]
snp_data["p"] = stats.norm.cdf(snp_data["z"])

In [201]:
snp_data.head()

,chr,pos,ref,alt,z,variant,se,beta,p
0,1,158934025,C,T,0.594883,1:158934025:C:T,1,0.594883,0.724039
1,1,158935661,G,A,-0.991344,1:158935661:G:A,1,-0.991344,0.160759
2,1,158935773,A,T,1.381962,1:158935773:A:T,1,1.381962,0.916508
3,1,158936157,CCATTAACTCAT,C,1.640093,1:158936157:CCATTAACTCAT:C,1,1.640093,0.949507
4,1,158936329,C,A,1.658588,1:158936329:C:A,1,1.658588,0.951401


In [202]:
# set abf to TRUE
df = snp_data
lbf, prob = abf(df.beta, df.se, W=0.04)
cs = get_cs(df.variant, prob, coverage=0.95)
cs_99 = get_cs(df.variant, prob, coverage=0.99)
df["lbf"] = lbf
df["prob"] = prob
df["cs"] = df.variant.isin(cs)
df["cs_99"] = df.variant.isin(cs_99)


In [203]:
lead_idx_snp = df["p"].idxmin()
df["lead_variant"] = False
df.at[lead_idx_snp, "lead_variant"] = True
lead_z = (df.beta / df.se).iloc[lead_idx_snp]
df['r'] = LD_data.iloc[:, lead_idx_snp]
df["t_dentist_s"] = ((df.beta / df.se) - df.r * lead_z) ** 2 / (1 - df.r ** 2)
df["t_dentist_s"] = np.where(df["t_dentist_s"] < 0, np.inf, df["t_dentist_s"])
df["t_dentist_s"].iloc[lead_idx_snp] = np.nan
df.loc[lead_idx_snp, "t_dentist_s"] = np.nan
df["nlog10p_dentist_s"] = sp.stats.chi2.logsf(df["t_dentist_s"], df=1) / -np.log(10)

/tmp/8118196.1.csg.q/ipykernel_45278/1561113221.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["t_dentist_s"].iloc[lead_idx_snp] = np.nan


In [204]:
df.head()

,chr,pos,ref,alt,z,variant,se,beta,p,lbf,prob,cs,cs_99,lead_variant,r,t_dentist_s,nlog10p_dentist_s
0,1,158934025,C,T,0.594883,1:158934025:C:T,1,0.594883,0.724039,-0.012805,0.000988,True,True,False,0.304558,3.448197,1.198454
1,1,158935661,G,A,-0.991344,1:158935661:G:A,1,-0.991344,0.160759,-0.000711,0.001000,True,True,False,0.004281,0.950339,0.481969
2,1,158935773,A,T,1.381962,1:158935773:A:T,1,1.381962,0.916508,0.017117,0.001018,True,True,False,0.004023,1.952946,0.789763
3,1,158936157,CCATTAACTCAT,C,1.640093,1:158936157:CCATTAACTCAT:C,1,1.640093,0.949507,0.032119,0.001034,True,True,False,0.023794,3.000830,1.079763
4,1,158936329,C,A,1.658588,1:158936329:C:A,1,1.658588,0.951401,0.033292,0.001035,True,True,False,0.012761,2.916960,1.057234


In [205]:
df["r2"] = df.r ** 2

# case_control = TRUE for effective samples
# if args.case_control:
#     df["n_eff_samples"] = df.n_samples * (df.n_cases / df.n_samples) * (1 - df.n_cases / df.n_samples)
# else:
#     df["n_eff_samples"] = df.n_samples

# manual set r2_threshold and nlog10p_dentist_s_threshold (according to R function)
r2_threshold = 0.6
nlog10p_dentist_s_threshold = 4.0
n_r2 = np.sum(df.r2 > r2_threshold)
print(f"n_r2={n_r2}")

n_dentist_s_outlier = np.sum(
    (df.r2 > r2_threshold) & (df.nlog10p_dentist_s > nlog10p_dentist_s_threshold)
)
print(f"n_dentist_s_outlier={n_dentist_s_outlier}")

max_pip = np.max(df.prob)
print(f"max_pip={max_pip}")

n_r2=22
n_dentist_s_outlier=21
max_pip=0.0013062877743510654


# slalom in R

In [209]:
rm(list=ls())
library(readr)
library(dplyr, warn.conflicts = FALSE)
library(vroom)
source("/home/rd2972/software/pecotmr/R/slalom.R")

In [210]:
test_data_path = "/home/rd2972/private_data/20240300_rss_qc_imputation/SLALOM/"
snp_data <- vroom(paste0(test_data_path, "z_LD_dentist_test_sum.stat.txt"))
LD_data <- as.matrix(vroom(paste0(test_data_path, "z_LD_dentist_test_LD.txt")))

In [211]:
snp_data$p_value <- pnorm(snp_data$z)

In [212]:
results <- slalom(LDmat = LD_data, zScore = snp_data$z)

In [213]:
head(results$data)

,original_z,prob,pvalue,outliers,nlog10p_dentist_s
,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>
1,0.5948835,0.0009883897,0.5519214,FALSE,1.1984537
2,-0.9913440,0.0010004157,0.3215176,FALSE,0.4819691
3,1.3819617,0.0010184110,0.1669835,FALSE,0.7897625
4,1.6400928,0.0010338040,0.1009859,FALSE,1.0797626
5,1.6585882,0.0010350177,0.0971988,FALSE,1.0572339
6,0.2938973,0.0009833181,0.7688364,FALSE,1.6361556


In [214]:
results$summary

$lead_pip_variant
[1] 203

$n_total
[1] 1000

$n_r2
[1] 22

$n_dentist_s_outlier
[1] 21

$fraction
[1] 0.9545455

$max_pip
[1] 0.001306288

$cs_95
  [1]  203  741  126   12  549  623 1000  895  770  733  696  737  202  467  705
 [16]  596  858  292  261  732   18  707   10  777  717  818  586  574  798  701
 [31]  527  237  430  312  191  800  459  511  725  151  159  941  719  452  185
 [46]  455  752  712  453  985  576  458  882  431  216  415  269  721  342  833
 [61]  509   86  532  691  491    5  433  358  249  490  403  413  411  566  410
 [76]  672  408    4  568  573  571  694  615  219    9  519  493  539  492  404
 [91]  497   32  468  209  486  412  489  398  460  538  483  200  482  481  462
[106]  432  480   47  595  474  476  267  610  470  471  692   54  295  270  289
[121]  287  575  201  291  643  311  641  636  881  673  971  214  860  817  826
[136]  600  552  518  256  405  735  740  284   46  545  671  823  283  548  118
[151]  627  257  603  528  506  658  835  670  822  898  850   77  758  803  172
[166]  567  893  629    3  123  579  689  757  699  764  106  597  148  824   45
[181]  828  809  189  734  743  951  171  262  704  612  435  698  372  614  272
[196]  749  630  598  602  611  718  789  290  484   71  761  715  242  684  706
[211]  887  557  876  769  739  601  768  125  760  593  621  533  744  693  753
[226]  208  246  211  205  212  220  918  901  473  846  466   89  599   75  944
[241]  807  819  522  305  703  865  303  987  485   80  451  814  613  617  530
[256]  856  332  606  781  178  181  184  224  112   83  892  448  771   66  215
[271]   23  979   93   98  928  368   59  103  132  713  878  869  880  282  501
[286]  890  864  885  243  886  140  891  302  799  830  258  514  578  943  584
[301]  119  488  648  537  747  500  745  157  952  281  348  180  187   33  788
[316]  188  558  285  585    2  351  879  357   22  697  359   69  336  334  349
[331]  279  278  935  668  843  277  356  417  499  577  365  327  439  280  465
[346]  167  784  328  683  396  343  271  832  352  975  179  748  366  708   15
[361]  729  910  350  637  840  274  361  994  335  642  841  782  353  307  915
[376]  104  487  323  338  293   72  367  313  982  695  360  583  355  164  873
[391]  425  100  101  102  108  345  521   95  339  254  273  309  347  836  988
[406]  147  320  393  989  851  711  319  321   96  572  397  937  337  524   76
[421]  346  866  505  107   49  779  276  105  776  680  674   99  245  244  325
[436]  854  690  589  526  169  422  896  868  175  176  230  195  170  378  134
[451]  196  251  114  115  756  128  661  494  182  192  322  131  594  924  516
[466]  174  116  308  588  121  229  400  416  122  324  133   97  218  110  197
[481]  508  401  472  587  520  529  129  127  190   58  301  534  109  117  111
[496]  523   50  221  240  536  475  513  113  838  884  210  654  461  507  504
[511]   60  810  168  563  722  857  976  512  250  570  569  515  767  238  296
[526]  677  207  260  135  158  990  142  428   19  141  143  146  390  751  816
[541]   92  454  531   94  496    7  144  820  561  582  720  847  389  145  247
[556]  370    8  562  234  380  498  232  236  682  231  646  362  996  388  540
[571]  905   79  727  235  544  227  228  385  554  802  547  387  676  804  223
[586]  222  688  772  386  375  139  160  551  801  376  194  553  186  656  736
[601]  120  525  252  377  154  645  383  382  560  791  149  162  663  138  831
[616]    1  746  542  150  165  156  503  556  331  631  543  371  233  845   74
[631]  626  456  628  960  437  363  364  778  904  440  163  842  947  255  414
[646]  193   78  795  927  438  632   13  436  624  591  774  502  259  263  443
[661]  265  759  444  605  213  639  152  863  559  153  421  479  622  961  959
[676]  844  136  535  590  177  564  647  667  926  608   90  932  619  916  950
[691]  968  625  909  974   82   48  609  783  620  991  933   81  797  956  253
[706]  972   64  903  678  973  354  775  939 

Note that the `t_dentist_s`, `pvalue` and `nlog10p_dentist_s` variables are consistent between the results in R and python for the variants. And the `max_pip`, `n_r2` and `n_dentist_s_outlier` are also consistent.